In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import re

from nltk import bigrams, trigrams, ngrams, word_tokenize

from string import punctuation

from collections import defaultdict

from itertools import chain # Con esto vamos a juntas todos los ngramas en un corpus...

In [2]:
df = pd.read_csv("train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4417 non-null   int64 
 1   Title       4416 non-null   object
 2   Opinion     4415 non-null   object
 3   Place       4417 non-null   object
 4   Gender      4417 non-null   object
 5   Age         4417 non-null   int64 
 6   Country     4417 non-null   object
 7   Date        4417 non-null   object
 8   Label       4417 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 310.7+ KB


In [3]:
# Hay unos valores nulos pero no coinciden... así que al unir Title y Opinion no se afectará el resultado...
print("Title: \n", df["Title"][df["Title"].isnull()])
print("Opinion: \n", df["Opinion"][df["Opinion"].isnull()])

Title: 
 2209    NaN
Name: Title, dtype: object
Opinion: 
 1142    NaN
3712    NaN
Name: Opinion, dtype: object


In [4]:
# Vamos a unir las columnas de texto en cuestión para la primera prueba...
# Crear una función para concatenar las columnas "Title" y "Opinion"...
def concatenate_columns(row):
    # Convertir ambos valores a cadenas, si son None o NaN, los convertirá en una cadena vacía...
    title = str(row["Title"]) if pd.notna(row["Title"]) else ""
    opinion = str(row["Opinion"]) if pd.notna(row["Opinion"]) else ""

    if (title != "") and (opinion != ""):
        return title + ". " + opinion
    elif (title == ""):
        return opinion
    elif (opinion == ""):
        return title

df["Texto"] = df.apply(concatenate_columns, axis=1)

In [5]:
# Resuelto...
print(df["Texto"][1142])
print(df["Texto"][2209])
print(df["Texto"][3712])

Lugar lleno de historia
De los lugares más bonitos de Guanajuato, a un costado del Teatro Juárez, visita obligatoria en Guanajuato
punto cental


In [6]:
# Eliminando las comillas dobles en el texto final, si existen...
df["Texto"] = df["Texto"].str.replace('"', '')

# Mostrar el resultado...
df["Texto"][0]

'No es para recomendarse. Las famosas momias de Guanajuato. El museo en sí es horrible, y no por lo que exhiben sino como lo exhiben. Mi abuelo me contó que hace 40 años para visitar las momias había que entrar al panteón pasear entre las tumbas y darle una propina al sepulturero que levantaba una pesada aldaba para después bajar por una escalera de caracol a la lúgubre cripta. Ahí los cuerpos alineados uno tras otro y a ambos lados de un estrecho pasillo había que pasar entre ellas sin cristales de por medio. Suena más escalofriante pero mucho más auténtico... lo debieron haber dejado así y limitar el número de personas que las pudieran visitar diariamente quizás unas 100 y haber cobrado más. Pero bueno se ve que el museo es una muy buena fuente de ingresos para el erario de esta bella ciudad. Recomiendo visitarlo sí y sólo si ya visitaron todas las demás atracciones y bellezas que ofrece Guanajuato si les queda tiempo pues esta bien.'

In [7]:
# Vamos a eliminar la columna Unnamed, que contenía Índices...
df.drop('Unnamed: 0', axis=1, inplace= True)

Sobre algunos conceptos mencionados en clases, en relación con el preprocesamiento del texto, surgió la idea de agragar las etiquetas <SOS>, <EOS> y <UNK>. Solo agregaremos <SOS> y <UNK>, ya que consideramos que agregar <EOS> crearía 2-gramas <EOS> <SOS> que no servirían de mucho... al menos en lo que a 2-gramas respecta. Igual se valorará que hacer más adelante... 

In [8]:
# Definir una función para agregar <SOS> delante de cada palabra que inicie una oración
def add_sos(text):
    # Expresión regular para encontrar el inicio de una oración
    modified_text = re.sub(r'(?<!\w)([A-Z][a-z]*\b)', r'startofsentence \1', text)
    return modified_text

# Aplicar la función a la columna de texto
df["Texto"] = df["Texto"].apply(add_sos)

In [9]:
# Eliminar los caracteres '<' y '>'
# punctuacion = punctuation.replace('<', '').replace('>', '')
punctuations = set(punctuation)
# punctuations

In [10]:
# Hasta aquí muy fácil... ahora vamos a ver que palabras son menos frecuentes y sustituirlas por <UNK>
# Pero primero hay que Tokenizar...

# Crear una función para tokenizar y eliminar signos de puntuación
def tokenize_and_remove_punctuation(text):
    # Convertir a minúsculas y tokenizar el texto
    tokens = word_tokenize(text.lower())
    
    # Eliminar signos de puntuación
    tokens = [word for word in tokens if word not in punctuations]
    
    return tokens

# Aplicar la función a la columna de texto modificado
df["Tokens"] = df["Texto"].apply(tokenize_and_remove_punctuation)

In [11]:
# Crear un defaultdict para contar palabras...
# Contar palabras nos da la medida de cual es el mínimo de ngramas que serán únicos...
# Sin embargo no es palabras lo que queremos contar, sino n-gramas...

word_count = defaultdict(int)

# Recorrer todos los tokens y contar las palabras
for tokens in df["Tokens"]:
    for token in tokens:
        word_count[token.lower()] += 1

# Mostrar los conteos de palabras
for word, count in word_count.items():
    print(f"{word}: {count}")

startofsentence: 16428
no: 2538
es: 4629
para: 2368
recomendarse: 1
las: 1886
famosas: 25
momias: 544
de: 10551
guanajuato: 1818
el: 4315
museo: 962
en: 4137
sí: 98
horrible: 11
y: 5976
por: 2065
lo: 1581
que: 5780
exhiben: 14
sino: 57
como: 780
mi: 210
abuelo: 1
me: 681
contó: 8
hace: 228
40: 18
años: 177
visitar: 588
había: 98
entrar: 210
al: 1083
panteón: 25
pasear: 43
entre: 100
tumbas: 4
darle: 24
una: 2336
propina: 76
sepulturero: 1
levantaba: 1
pesada: 15
aldaba: 1
después: 113
bajar: 54
escalera: 37
caracol: 3
a: 2803
la: 7863
lúgubre: 5
cripta: 2
ahí: 321
los: 2011
cuerpos: 123
alineados: 1
uno: 335
tras: 4
otro: 107
ambos: 3
lados: 14
un: 3235
estrecho: 31
pasillo: 6
pasar: 204
ellas: 21
sin: 514
cristales: 6
medio: 33
suena: 3
más: 875
escalofriante: 6
pero: 1122
mucho: 436
auténtico: 4
...: 306
debieron: 2
haber: 51
dejado: 4
así: 232
limitar: 2
número: 5
personas: 205
pudieran: 5
diariamente: 1
quizás: 11
unas: 94
100: 35
cobrado: 2
bueno: 145
se: 1902
ve: 103
muy: 2369
bu

In [28]:
# Démosle ya un vistazo al DataFrame...
df

,Title,Opinion,Place,Gender,Age,Country,Date,Label,Texto,Tokens
0,"""No es para recomendarse""","""Las famosas momias de Guanajuato. El museo en...",Museo de las Momias,Male,48,México,02/07/2013,0,startofsentence No es para recomendarse. start...,"[startofsentence, no, es, para, recomendarse, ..."
1,"""Es un mercado tradicional, no es nada del otr...","""Allí puedes encontrar comida o souvenirs. Es ...",Mercado Hidalgo,Male,52,México,06/09/2016,0,"startofsentence Es un mercado tradicional, no ...","[startofsentence, es, un, mercado, tradicional..."
2,"""No visitar""","""Es tardado entrar al museo, una vez dentro no...",Museo de las Momias,Male,42,México,25/12/2017,0,startofsentence No visitar. startofsentence Es...,"[startofsentence, no, visitar, startofsentence..."
3,"""No le veo atractivo. A ver semejantes cacharros""","""Entiendo que a muchos les cautive o llamé la ...",Museo de las Momias,Female,24,México,14/08/2013,0,startofsentence No le veo atractivo. startofse...,"[startofsentence, no, le, veo, atractivo, star..."
4,"""Nada fuera de lo común""","""La fruta y verdura son poco frescos, al igual...",Mercado Hidalgo,Female,19,México,01/08/2017,0,startofsentence Nada fuera de lo común. starto...,"[startofsentence, nada, fuera, de, lo, común, ..."
...,...,...,...,...,...,...,...,...,...,...
4412,"""Beautiful architecture""","""This university is famous because of its beau...",Universidad de Guanajuato,Male,24,México,21/04/2014,1,startofsentence Beautiful architecture. starto...,"[startofsentence, beautiful, architecture, sta..."
4413,"""Imponente la universidad, TODO guanajuato es ...","""Lo que no me gusto fue la comida, pero es muy...",Universidad de Guanajuato,Female,44,México,05/07/2015,1,"startofsentence Imponente la universidad, TODO...","[startofsentence, imponente, la, universidad, ..."
4414,Lugar magico,Esta excelente para pasar a sentarse unos minu...,Jardín de la Unión,Male,-1,México,2018,1,startofsentence Lugar magico. startofsentence ...,"[startofsentence, lugar, magico, startofsenten..."
4415,"""¡Su fachada es hermosa!""","""No tuvimos la fortuna de entrar a verlo, ya q...",Teatro Juárez,Female,47,México,22/10/2012,1,¡startofsentence Su fachada es hermosa!. start...,"[¡startofsentence, su, fachada, es, hermosa, s..."


In [40]:
# Función para generar n-gramas a partir de una lista de tokens
def generate_ngrams(text, n):
    # Tokenizar el texto
    # tokens = word_tokenize(text.lower())
    
    # Generar los n-gramas
    n_grams = list(ngrams(text, n))
    
    return n_grams

In [46]:
# Crear las nuevas columnas con los n-gramas...
for n in range(2,6):
    df[f'n{n}-grams'] = df['Tokens'].apply(lambda x: generate_ngrams(x, n))

In [47]:
df

,Title,Opinion,Place,Gender,Age,Country,Date,Label,Texto,Tokens,n2-grams,n3-grams,n4-grams,n5-grams
0,"""No es para recomendarse""","""Las famosas momias de Guanajuato. El museo en...",Museo de las Momias,Male,48,México,02/07/2013,0,startofsentence No es para recomendarse. start...,"[startofsentence, no, es, para, recomendarse, ...","[(startofsentence, no), (no, es), (es, para), ...","[(startofsentence, no, es), (no, es, para), (e...","[(startofsentence, no, es, para), (no, es, par...","[(startofsentence, no, es, para, recomendarse)..."
1,"""Es un mercado tradicional, no es nada del otr...","""Allí puedes encontrar comida o souvenirs. Es ...",Mercado Hidalgo,Male,52,México,06/09/2016,0,"startofsentence Es un mercado tradicional, no ...","[startofsentence, es, un, mercado, tradicional...","[(startofsentence, es), (es, un), (un, mercado...","[(startofsentence, es, un), (es, un, mercado),...","[(startofsentence, es, un, mercado), (es, un, ...","[(startofsentence, es, un, mercado, tradiciona..."
2,"""No visitar""","""Es tardado entrar al museo, una vez dentro no...",Museo de las Momias,Male,42,México,25/12/2017,0,startofsentence No visitar. startofsentence Es...,"[startofsentence, no, visitar, startofsentence...","[(startofsentence, no), (no, visitar), (visita...","[(startofsentence, no, visitar), (no, visitar,...","[(startofsentence, no, visitar, startofsentenc...","[(startofsentence, no, visitar, startofsentenc..."
3,"""No le veo atractivo. A ver semejantes cacharros""","""Entiendo que a muchos les cautive o llamé la ...",Museo de las Momias,Female,24,México,14/08/2013,0,startofsentence No le veo atractivo. startofse...,"[startofsentence, no, le, veo, atractivo, star...","[(startofsentence, no), (no, le), (le, veo), (...","[(startofsentence, no, le), (no, le, veo), (le...","[(startofsentence, no, le, veo), (no, le, veo,...","[(startofsentence, no, le, veo, atractivo), (n..."
4,"""Nada fuera de lo común""","""La fruta y verdura son poco frescos, al igual...",Mercado Hidalgo,Female,19,México,01/08/2017,0,startofsentence Nada fuera de lo común. starto...,"[startofsentence, nada, fuera, de, lo, común, ...","[(startofsentence, nada), (nada, fuera), (fuer...","[(startofsentence, nada, fuera), (nada, fuera,...","[(startofsentence, nada, fuera, de), (nada, fu...","[(startofsentence, nada, fuera, de, lo), (nada..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,"""Beautiful architecture""","""This university is famous because of its beau...",Universidad de Guanajuato,Male,24,México,21/04/2014,1,startofsentence Beautiful architecture. starto...,"[startofsentence, beautiful, architecture, sta...","[(startofsentence, beautiful), (beautiful, arc...","[(startofsentence, beautiful, architecture), (...","[(startofsentence, beautiful, architecture, st...","[(startofsentence, beautiful, architecture, st..."
4413,"""Imponente la universidad, TODO guanajuato es ...","""Lo que no me gusto fue la comida, pero es muy...",Universidad de Guanajuato,Female,44,México,05/07/2015,1,"startofsentence Imponente la universidad, TODO...","[startofsentence, imponente, la, universidad, ...","[(startofsentence, imponente), (imponente, la)...","[(startofsentence, imponente, la), (imponente,...","[(startofsentence, imponente, la, universidad)...","[(startofsentence, imponente, la, universidad,..."
4414,Lugar magico,Esta excelente para pasar a sentarse unos minu...,Jardín de la Unión,Male,-1,México,2018,1,startofsentence Lugar magico. startofsentence ...,"[startofsentence, lugar, magico, startofsenten...","[(startofsentence, lugar), (lugar, magico), (m...","[(startofsentence, lugar, magico), (lugar, mag...","[(startofsentence, lugar, magico, startofsente...","[(startofsentence, lugar, magico, startofsente..."
4415,"""¡Su fachada es hermosa!""","""No tuvimos la fortuna de entrar a verlo, ya q...",Teatro Juárez,Female,47,México,22/10/2012,1,¡startofsentence Su fachada es hermosa!. start...,"[¡startofsentence, su, fachada, es, hermosa, s...","[(¡startof

In [52]:
# Ahora si vamos a contar n-gramas...

word_count = defaultdict(int)

# Recorrer todos los tokens y contar las palabras
for n_grams in df["n2-grams"]:
    for n_gram in n_grams:
        word_count[n_gram] += 1

# Mostrar los conteos de palabras
# for word, count in word_count.items():
#     print(f"{word}: {count}")

# Convertir el diccionario en un DataFrame
df_word_count = pd.DataFrame(list(word_count.items()), columns=['word', 'count'])

df_word_count.sort_values(by='count', ascending=False, inplace=True)
df_word_count

,word,count
422,"(de, la)",1786
10,"(startofsentence, guanajuato)",1739
9,"(de, startofsentence)",1405
423,"(la, ciudad)",966
175,"(startofsentence, es)",957
...,...,...
31730,"(alguna, historia)",1
31731,"(historia, estaría)",1
31732,"(que, cuenten)",1
31733,"(sobre, esas)",1
